# MINIST XGBOOST 

This is a model with 92% accuracy, after solving the following issue , it should be better

####  Crash after setting ntree_limit= model.best_ntree_limit



1. __Business Understanding__: Using MINIST dataset to train model , then recognize digit image
2. __Analytic Approach:__ XGBOOST
3. __Data Requirement:__ using minist data [Kaggle digit recognizer](https://www.kaggle.com/c/digit-recognizer/data)
4. __Data Collection:__ using pandas, read from csv file
5. __Data Understanding:__ using pandas and visulizatio seaborn, it's easy, no complex featurs, only pixels(724)
6. __Data Preparation:__ pandas , no need more effort 
7. __Modeling__: xgboost, scikit-learn, GridSearchCV
8. __Evaluation__: scikit-learn, confusion matrix, F1 Score, ROC/AUC

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

## Data Collection

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Data Understanding

In [5]:
print('Train shape is:',train.shape)
print('Test shape is:', test.shape)

Train shape is: (42000, 785)
Test shape is: (28000, 784)


In [6]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Data Preparation

#### Get Features and labels

In [8]:
X = train.iloc[:, 1:]

In [9]:
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y = train.iloc[:, 0]

In [11]:
y.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

# Modeling

#### Split train set and validation set

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [13]:
print("X_train shape is:", X_train.shape)
print("X_val shape is:", X_val.shape)
print("y_train shape is:", y_train.shape)
print("y_val shape is:", y_val.shape)

X_train shape is: (29400, 784)
X_val shape is: (12600, 784)
y_train shape is: (29400,)
y_val shape is: (12600,)


#### xgboost matrix creation

In [14]:
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_val   = xgb.DMatrix(X_val, label=y_val)
xgb_test  = xgb.DMatrix(test)

#### specify parameters

In [17]:
params={
'booster':'gbtree',
'objective': 'multi:softmax', #multi classification
'num_class':10, # classes number as this is multi classification
'gamma':0.1,  # prunching parameter, normally , 0.1, 0.2, larger , less prunching
'max_depth':12, # depth of tree, bigger tree will lead to overfitting
'lambda':2,  #  model complexity , L2 regularization, bigger -> less overfitting
'subsample':0.7, # sampling randomly
'colsample_bytree':0.7, # column sampling when tree creation
'min_child_weight':3, 

# ???
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
# smaller, easier overfitting
    
#Minimum sum of instance weight (hessian) needed in a child. 
#If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight,
#then the building process will give up further partitioning. 
#In linear regression task, this simply corresponds to minimum number of instances needed to be in each node. 
#The larger min_child_weight is, the more conservative the algorithm will be.    
'silent':0 ,# running infor , set to : 0.
'eta': 0.007, # like learning rate
'seed':1000,
'nthread':7,# cpu threads
'tree_method': 'gpu_hist' # Use GPU accelerated algorithm    
#'eval_metric': 'auc'
}


#### specify validations set to watch performance

In [18]:
watchlist = [(xgb_train, 'train'), (xgb_val, 'eval')]

#num_iteration = 1200
num_iteration = 10

#### Train

In [19]:
%%time
model = xgb.train(params, xgb_train, num_boost_round= num_iteration, evals=watchlist, early_stopping_rounds=100)

[0]	train-merror:0.087891	eval-merror:0.124603
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 100 rounds.
[1]	train-merror:0.065646	eval-merror:0.098571
[2]	train-merror:0.058129	eval-merror:0.089841
[3]	train-merror:0.052891	eval-merror:0.080873
[4]	train-merror:0.050408	eval-merror:0.078492
[5]	train-merror:0.047245	eval-merror:0.074762
[6]	train-merror:0.045612	eval-merror:0.073095
[7]	train-merror:0.044184	eval-merror:0.07246
[8]	train-merror:0.043163	eval-merror:0.070317
[9]	train-merror:0.042381	eval-merror:0.070317
Wall time: 12 s


In [20]:
model.save_model('minist_xgboost.model')
print('Best best_ntree_limit', model.best_ntree_limit)

Best best_ntree_limit 9


# Evaluation

In [21]:
%%time
preds = model.predict(xgb_test, ntree_limit= model.best_ntree_limit)

Wall time: 175 ms


In [22]:
preds_df = pd.DataFrame(preds)
preds_df['ImageId'] = preds_df.index
preds_df['Label']=preds_df.iloc[:,0].astype(int)
preds_df.drop(0, axis=1, inplace=True)
preds_df.to_csv('xgb_submission.csv', index=False)

In [23]:
bst2 = xgb.Booster(model_file='minist_xgboost.model.backup')

[13:04:12] C:\Users\Administrator\Desktop\xgboost\src\learner.cc:362: Parameter 'predictor' has been recovered from the saved model. It will be set to 'gpu_predictor' for prediction. To override the predictor behavior, explicitly set 'predictor' parameter as follows:
  * Python package: bst.set_param('predictor', [new value])
  * R package:      xgb.parameters(bst) <- list(predictor = [new value])
  * JVM packages:   bst.setParam("predictor", [new value])


####  Crash after setting ntree_limit= model.best_ntree_limit

In [ ]:
preds2 = bst2.predict(xgb_test)

In [25]:
preds_df = pd.DataFrame(preds2)
preds_df['ImageId'] = preds_df.index + 1
preds_df['Label']=preds_df.iloc[:,0].astype(int)
preds_df.drop(0, axis=1, inplace=True)
preds_df.to_csv('xgb_submission2.csv', index=False)